In [193]:
import sys
sys.path.insert(0, './unwiki')
import unwiki
import re
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import pickle
from collections import Iterable

from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
import ner
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn import metrics

In [154]:
with open('../wiki_definitions_improved.txt', 'r') as wiki_f:
    wiki = wiki_f.readlines()

In [162]:
title, section, defini = wiki[1].split('-#-%-')
print(title)
unwiki.loads(st)

Astronaut  


' aboard Freedom 7\n\nThe criteria for what constitutes human spaceflight vary. The Fédération Aéronautique Internationale (FAI) Sporting Code for astronautics recognizes only flights that exceed an altitude of . In the United States, professional, military, and commercial astronauts who travel above an altitude of  are awarded astronaut wings.\n\n, a total of 552 people from 36 countries have reached  or more in altitude, of which 549 reached low Earth orbit or beyond.\nOf these, 24 people have traveled beyond low Earth orbit, either to lunar orbit, the lunar surface, or, in one case, a loop around the Moon. Three of the 24–Jim Lovell, John Young and Eugene Cernan–did so twice. The three current astronauts who have flown without reaching low Earth orbit are spaceplane pilots Joe Walker, Mike Melvill, and Brian Binnie, who participated in suborbital missions.\n\n, under the U.S. definition, 558 people qualify as having reached space, above  altitude. Of eight X-15 pilots who exceeded  

In [89]:
# Get data and train the Sentence tokenizer
text = ''
for i in range(250):
    text += unwiki.loads(eval(wiki[i].split('---')[2]))

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [163]:
title, section, defin = wiki[210].split('-#-%-')
dclean = unwiki.loads(eval(defin))
tokenizer.tokenize(dclean)    

['A normed vector space is a pair _inline_math_ where _inline_math_ is a vector space and _inline_math_ a norm on _inline_math_.',
 'A seminormed vector space is a pair _inline_math_ where _inline_math_ is a vector space and _inline_math_ a seminorm on _inline_math_.',
 'We often omit _inline_math_ or _inline_math_ and just write _inline_math_ for a space if it is clear from the context what (semi) norm we are using.',
 'In a more general sense, a vector norm can be taken to be any real-valued function that satisfies the three properties above.',
 'A useful variation of the triangle inequality is\n:_inline_math_ for any vectors x and y.',
 'This also shows that a vector norm is a continuous function.',
 'Note that property 2 depends on a choice of norm _inline_math_ on the field of scalars.',
 'When the scalar field is _inline_math_ (or more generally a subset of _inline_math_), this is usually taken to be the ordinary absolute value, but other choices are possible.',
 'For example, fo

In [164]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [165]:
def_lst = []
for i in range(len(wiki)):
    try:
        title, section, defin_raw = wiki[i].split('-#-%-')
        defin_all = unwiki.loads(eval(defin_raw))
        for d in tokenizer.tokenize(defin_all):
            if title.lower().strip() in d.lower():
                pos_tokens = pos_tag(word_tokenize(d))
                def_ner = ner.bio_tag.bio_tagger(title.strip().split(), pos_tokens)
                other_ner = [((d[0],d[1]),d[2]) for d in def_ner]
                tmp_dict = {'title': title,
                           'section': section,
                           'defin': d,
                           'ner': other_ner}
                def_lst.append(tmp_dict)
    except ValueError:
        print('parsing error')

In [167]:
s = def_lst[414]['ner']
s

[(('As', 'IN'), 'O'),
 (('with', 'IN'), 'O'),
 (('dharma', 'NN'), 'B-DFNDUM'),
 ((',', ','), 'O'),
 (('the', 'DT'), 'O'),
 (('word', 'NN'), 'O'),
 (('adharma', 'NN'), 'O'),
 (('includes', 'VBZ'), 'O'),
 (('and', 'CC'), 'O'),
 (('implies', 'NNS'), 'O'),
 (('many', 'JJ'), 'O'),
 (('ideas', 'NNS'), 'O'),
 ((';', ':'), 'O'),
 (('in', 'IN'), 'O'),
 (('common', 'JJ'), 'O'),
 (('parlance', 'NN'), 'O'),
 ((',', ','), 'O'),
 (('adharma', 'NN'), 'O'),
 (('means', 'NNS'), 'O'),
 (('that', 'IN'), 'O'),
 (('which', 'WDT'), 'O'),
 (('is', 'VBZ'), 'O'),
 (('against', 'IN'), 'O'),
 (('nature', 'NN'), 'O'),
 ((',', ','), 'O'),
 (('immoral', 'JJ'), 'O'),
 ((',', ','), 'O'),
 (('unethical', 'JJ'), 'O'),
 ((',', ','), 'O'),
 (('wrong', 'JJ'), 'O'),
 (('or', 'CC'), 'O'),
 (('unlawful', 'JJ'), 'O'),
 (('.', '.'), 'O')]

In [168]:
class NamedEntityChunker(ChunkParserI):
    def __init__(self, train_sents, **kwargs):
        assert isinstance(train_sents, Iterable)
 
        self.feature_detector = features
        self.tagger = ClassifierBasedTagger(
            train=train_sents,
            feature_detector=features,
            **kwargs)
    def parse(self, tagged_sent):
        chunks = self.tagger.tag(tagged_sent)
 
        # Transform the result from [((w1, t1), iob1), ...] 
        # to the preferred list of triplets format [(w1, t1, iob1), ...]
        iob_triplets = [(w, t, c) for ((w, t), c) in chunks]
 
        # Transform the list of triplets to nltk.Tree format
        return conlltags2tree(iob_triplets)       
        
training_samples = [d['ner'] for d in def_lst[:int(len(def_lst) * 0.9)]]
test_samples = [d['ner'] for d in def_lst[int(len(def_lst) * 0.9):]]
 
print("#training samples = %s" % len(training_samples) )   # training samples = 55809
print("#test samples = %s" % len(test_samples))            # test samples = 6201
 

#training samples = 12056
#test samples = 1340


In [170]:
#train takes a long time
%time chunker = NamedEntityChunker(training_samples)

CPU times: user 1min 48s, sys: 1.11 s, total: 1min 49s
Wall time: 1min 51s


In [174]:
print(chunker.parse(pos_tag(word_tokenize("We define a Banach space as a complete vector space."))))

(S
  We/PRP
  define/VBP
  a/DT
  (DFNDUM Banach/NNP space/NN)
  as/IN
  a/DT
  (DFNDUM complete/JJ vector/NN space/NN)
  ./.)


In [198]:
test_samples[range(2,5)]

TypeError: list indices must be integers or slices, not range

In [208]:
def prepare_for_metrics(int_range):
    '''
    `int_range` is an integer range
    NEEDS A TEST_SAMPLES VARIABLE CREATED WHEN SPLITTING THE 
    TRAINING AND TESTING DATA
    Returns two vectors ready to be used in the 
    metrics classification function
    '''
    if isinstance(int_range, int):
        int_range = [int_range]
    y_true = []
    y_pred = []
    for i in int_range:
        sample = test_samples[i]
        sm = [s[0] for s in sample]
        y_true_tmp = [s[1] for s in sample]
        predicted = [v[2] for v in tree2conlltags(chunker.parse(sm))]
        y_true += y_true_tmp
        y_pred += predicted
    return y_true, y_pred

In [207]:
y_true, predicted = prepare_for_metrics(range(len(test_samples)))
print(metrics.classification_report(y_true, predicted))

              precision    recall  f1-score   support

    B-DFNDUM       0.30      0.73      0.43      1260
    I-DFNDUM       0.29      0.75      0.42      1232
           O       0.99      0.90      0.94     43929

   micro avg       0.89      0.89      0.89     46421
   macro avg       0.52      0.80      0.59     46421
weighted avg       0.95      0.89      0.91     46421



In [201]:
isinstance(45,int)

True

In [83]:
print(tokenizer._params.abbrev_types)

{'etc', 'dr', 'al', 'ice', 'pl', 'u.s', 'hz', 'i.e', 'uk', 'e.g', 'rna'}


In [121]:
def features(tokens, index, history):
    """
    `tokens`  = a POS-tagged sentence [(w1, t1), ...]
    `index`   = the index of the token we want to extract features for
    `history` = the previous predicted IOB tags
    """
 
    # init the stemmer
    stemmer = SnowballStemmer('english')
 
    # Pad the sequence with placeholders
    tokens = [('[START2]', '[START2]'), ('[START1]', '[START1]')] +\
    list(tokens) + [('[END1]', '[END1]'), ('[END2]', '[END2]')]
    history = ['[START2]', '[START1]'] + list(history)
 
    # shift the index with 2, to accommodate the padding
    index += 2
 
    word, pos = tokens[index]
    prevword, prevpos = tokens[index - 1]
    prevprevword, prevprevpos = tokens[index - 2]
    nextword, nextpos = tokens[index + 1]
    nextnextword, nextnextpos = tokens[index + 2]
    previob = history[index - 1]
    contains_dash = '-' in word
    contains_dot = '.' in word
    allascii = all([True for c in word if c in string.ascii_lowercase])
 
    allcaps = word == word.capitalize()
    capitalized = word[0] in string.ascii_uppercase
 
    prevallcaps = prevword == prevword.capitalize()
    prevcapitalized = prevword[0] in string.ascii_uppercase
 
    nextallcaps = prevword == prevword.capitalize()
    nextcapitalized = prevword[0] in string.ascii_uppercase
 
    return {
        'word': word,
        'lemma': stemmer.stem(word),
                'pos': pos,
        'all-ascii': allascii,
 
        'next-word': nextword,
        'next-lemma': stemmer.stem(nextword),
        'next-pos': nextpos,
 
        'next-next-word': nextnextword,
        'nextnextpos': nextnextpos,
 
        'prev-word': prevword,
        'prev-lemma': stemmer.stem(prevword),
        'prev-pos': prevpos,
 
        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,
 
        'prev-iob': previob,
 
        'contains-dash': contains_dash,
        'contains-dot': contains_dot,
 
        'all-caps': allcaps,
        'capitalized': capitalized,
 
        'prev-all-caps': prevallcaps,
        'prev-capitalized': prevcapitalized,
 
        'next-all-caps': nextallcaps,
        'next-capitalized': nextcapitalized,
    }
 